In [ ]:
%tensorflow_version 2.x

In [ ]:
!pip install -q sklearn

In [49]:
#import libraries
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

In [50]:
#read data and preprocess 🤷‍♂️👀
df_train = pd.read_csv('/content/train1.csv')
df_eval = pd.read_csv('/content/eval1.csv')
y_train = df_train.pop('HeartDisease')
y_eval = df_eval.pop('HeartDisease')

#divide different columns because of their data_type
CATEGORICAL_COLUMNS = ['Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS', 'RestingECG', 'MaxHR', 'ExerciseAngina', 'ST_Slope']
NUMERIC_COLUMNS = ['Age', 'Oldpeak']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = df_train[feature_name].unique() #print all the unique values in each columns
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
    
for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype= tf.float64))


In [51]:
#create input function that helps train the model in baches and or epochs 🤷‍♂️👀
def make_input_function(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def input(): # this function will be returned in the function
        data_set = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            data_set = data_set.shuffle(1000) #randomize the order of data
        data_set = data_set.batch(batch_size).repeat(num_epochs)   #split the data into sets(batches) of 32 and repeat the process 10 times(num of epochs)
        return data_set  #returns a batch of the data_set
    return input

train_input_function = make_input_function(df_train, y_train)  #change the format of the dataset into one that we can feed the model
eval_input_function = make_input_function(df_eval, y_eval, num_epochs= 1, shuffle= False)  #this is for testing the model


In [54]:
#Initialization of the model (i.e:- Linear_Regression)
linear_estimator = tf.estimator.LinearClassifier(feature_columns= feature_columns)

linear_estimator.train(train_input_function)    #trainning
result = linear_estimator.evaluate(eval_input_function)  #testing

clear_output()    #clear console outputs
print(result['accuracy'])       #this will show the accuracy of the model's prediction on the data_set for evaluatio
#this will vary since the testing data_set is on shuffle

0.7928994


In [ ]:
#if i had better data, the model's accuracy could probably be higher